In [1]:
import numpy as np
import numba as nb
import scipy as sp

In [2]:
def get_lmoments(x, nmom=5) -> np.ndarray:
    try:
        x = np.asarray(x, dtype=np.float64)
        n = x.shape[0]
        x.sort(axis=0)
    except ValueError:
        raise ValueError("Input data to estimate L-moments must be numeric.")

    if nmom <= 0 or nmom > 5:
        raise ValueError("Invalid number of sample L-moments")

    if n < nmom:
        raise ValueError("Insufficient length of data for specified nmoments")

    # First L-moment

    l1 = np.sum(x, axis=0) / sp.special.comb(n, 1, exact=True)

    if nmom == 1:
        return l1

    # Second L-moment

    comb1 = range(n)
    coefl2 = 0.5 / sp.special.comb(n, 2, exact=True)
    sum_xtrans = sum([(comb1[i] - comb1[n - i - 1]) * x[i] for i in range(n)])
    l2 = coefl2 * sum_xtrans

    if nmom == 2:
        return np.array([l1, l2])

    # Third L-moment

    comb3 = [sp.special.comb(i, 2, exact=True) for i in range(n)]
    coefl3 = 1.0 / 3.0 / sp.special.comb(n, 3, exact=True)
    sum_xtrans = sum(
        [
            (comb3[i] - 2 * comb1[i] *
             comb1[n - i - 1] + comb3[n - i - 1]) * x[i]
            for i in range(n)
        ]
    )
    l3 = coefl3 * sum_xtrans / l2

    if nmom == 3:
        return np.array([l1, l2, l3])

    # Fourth L-moment

    comb5 = [sp.special.comb(i, 3, exact=True) for i in range(n)]
    coefl4 = 0.25 / sp.special.comb(n, 4, exact=True)
    sum_xtrans = sum(
        [
            (
                comb5[i]
                - 3 * comb3[i] * comb1[n - i - 1]
                + 3 * comb1[i] * comb3[n - i - 1]
                - comb5[n - i - 1]
            )
            * x[i]
            for i in range(n)
        ]
    )
    l4 = coefl4 * sum_xtrans / l2

    if nmom == 4:
        return np.array([l1, l2, l3, l4])

    # Fifth L-moment

    comb7 = [sp.special.comb(i, 4, exact=True) for i in range(n)]
    coefl5 = 0.2 / sp.special.comb(n, 5, exact=True)
    sum_xtrans = sum(
        [
            (
                comb7[i]
                - 4 * comb5[i] * comb1[n - i - 1]
                + 6 * comb3[i] * comb3[n - i - 1]
                - 4 * comb1[i] * comb5[n - i - 1]
                + comb7[n - i - 1]
            )
            * x[i]
            for i in range(n)
        ]
    )
    l5 = coefl5 * sum_xtrans / l2

    return np.array([l1, l2, l3, l4, l5])

In [3]:
@nb.njit(nb.float64[:, :](nb.float64[:, :, :], nb.int8), parallel=True, cache=False)
def get_lmoments_jit(x: np.ndarray, nmom: int = 5) -> np.ndarray:

    def sort_array_with_axis(x: np.ndarray, axis: int) -> np.ndarray:
        for i in nb.prange(x.shape[axis]):
            x[i] = np.sort(x[i])
        return x

    x = np.asarray(x, dtype=np.float64)
    n = x.shape[0]

    x = sort_array_with_axis(x, 0)

    sum_xtrans = np.empty_like(x[0])

    def comb(n, r):
        if r < 0 or r > n:
            return 0
        if r == 0 or r == n:
            return 1
        c = 1
        for i in range(1, r + 1):
            c = c * (n - i + 1) // i
        return c

    # First L-moment

    l1 = np.sum(x, axis=0) / comb(n, 1)

    if nmom == 1:
        return l1

    # Second L-moment

    comb1 = np.arange(n)
    coefl2 = 0.5 / comb(n, 2)
    sum_xtrans[:] = 0
    for i in nb.prange(n):
        sum_xtrans += (comb1[i] - comb1[n - 1 - i]) * x[i]
    l2 = coefl2 * sum_xtrans

    if nmom == 2:
        return l2

    # Third L-moment

    comb3 = np.zeros(n)
    for i in range(n):
        comb3[i] = comb(i, 2)
    coefl3 = 1.0 / 3.0 / comb(n, 3)
    sum_xtrans[:] = 0
    for i in nb.prange(n):
        sum_xtrans += (
            comb3[i] - 2 * comb1[i] * comb1[n - 1 - i] + comb3[n - 1 - i]
        ) * x[i]
    l3 = coefl3 * sum_xtrans / l2

    if nmom == 3:
        return l3

    # Fourth L-moment

    comb5 = np.zeros(n)
    for i in range(n):
        comb5[i] = comb(i, 3)
    coefl4 = 0.25 / comb(n, 4)
    sum_xtrans[:] = 0
    for i in nb.prange(n):
        sum_xtrans += (
            comb5[i]
            - 3 * comb3[i] * comb1[n - 1 - i]
            + 3 * comb1[i] * comb3[n - 1 - i]
            - comb5[n - 1 - i]
        ) * x[i]
    l4 = coefl4 * sum_xtrans / l2

    if nmom == 4:
        return l4

    # Fifth L-moment

    comb7 = np.zeros(n)
    for i in range(n):
        comb7[i] = comb(i, 4)
    coefl5 = 0.2 / comb(n, 5)
    sum_xtrans[:] = 0
    for i in nb.prange(n):
        sum_xtrans += (
            comb7[i]
            - 4 * comb5[i] * comb1[n - 1 - i]
            + 6 * comb3[i] * comb3[n - 1 - i]
            - 4 * comb1[i] * comb5[n - 1 - i]
            + comb7[n - 1 - i]
        ) * x[i]

    l5 = coefl5 * sum_xtrans / l2

    return l5

In [4]:
testdata = np.random.rand(192,5000,6000)

In [5]:
get_lmoments_jit(testdata,nmom=5)

array([[-0.54414214, -0.1074772 , -1.82092192, ...,  0.28764048,
        -0.56621076, -0.28623747],
       [ 1.02147579, -2.77615615, -0.72437503, ..., -0.04550304,
        -0.07638821,  0.07921786],
       [ 0.46787536,  0.09134959,  0.10030997, ...,  1.4830951 ,
        -4.96094538,  0.49368232],
       ...,
       [-0.27754264, -0.97279984, -3.58972223, ..., -0.47204029,
        -0.86646677, -1.28770446],
       [ 1.02853148,  0.53884745,  0.40325154, ..., -2.70459041,
        -0.24387141, -0.24658014],
       [-0.73614493, -1.02990223, -0.5142389 , ...,  2.08309844,
         0.10788515, -0.5330767 ]])

In [6]:
get_lmoments(testdata,nmom=5)[4]

array([[ 0.06031907,  0.05563109,  0.02514131, ..., -0.04971768,
        -0.06495855, -0.04118095],
       [ 0.14051047,  0.08278678,  0.04461501, ..., -0.06795971,
        -0.06786259, -0.07426363],
       [ 0.11131084,  0.08284029,  0.06698979, ..., -0.0616999 ,
        -0.05588552, -0.05673419],
       ...,
       [ 0.10966561,  0.03787215,  0.04235266, ..., -0.07925664,
        -0.06688003, -0.05918676],
       [ 0.0958274 ,  0.0723249 ,  0.07240785, ..., -0.09028584,
        -0.09793688, -0.11064256],
       [ 0.08297004,  0.04136901,  0.04435818, ..., -0.07223899,
        -0.11205063, -0.10134773]])

In [7]:
%timeit get_lmoments_jit(testdata,nmom=5)

1min 32s ± 3.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit get_lmoments(testdata,nmom=5)

5min 15s ± 20.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
get_lmoments_jit.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function get_lmoments_jit, 
/tmp/ipykernel_1969/4252686631.py (1)  


Parallel loop listing for  Function get_lmoments_jit, /tmp/ipykernel_1969/4252686631.py (1) 
-----------------------------------------------------------------------------------------|loop #ID
@nb.njit(nb.float64[:, :](nb.float64[:, :, :], nb.int8), parallel=True, cache=False)     | 
def get_lmoments_jit(x: np.ndarray, nmom: int = 5) -> np.ndarray:                        | 
                                                                                         | 
    def sort_array_with_axis(x: np.ndarray, axis: int) -> np.ndarray:                    | 
        for i in nb.prange(x.shape[axis]):-----------------------------------------------| #13
            x[i] = np.sort(x[i])                                                         | 
        return x                                                                         | 
                                                      